In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

molecules = pd.read_csv('molecules_dataset_cleared.csv')
X = molecules.drop(columns=['id', 'SMILES', 'SELFIES', 'ACCUMULATION'])
Y = (molecules['ACCUMULATION'] > 500).astype(int)

display(X.shape)
display(Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=91)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

(209, 200)

(209,)

In [8]:
from sklearn.model_selection import GridSearchCV

def my_grid_search(model, param_grid, X_train, Y_train, X_test, Y_test):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train, Y_train)
    print(grid_search.best_params_)
    print(f'Train best accuracy: {grid_search.best_score_}')
    best_model = grid_search.best_estimator_
    print(f'Test accuracy: {best_model.score(X_test, Y_test)}')
    return grid_search

In [9]:
def important_features(weights, ft_names, percentage=0.9):
    non_zero_weights = [[ft_names[i], weights[i]] for i in range(len(weights)) if weights[i] != 0]
    total_weight = sum([abs(w[1]) for w in non_zero_weights])
    non_zero_weights.sort(key=lambda x: abs(x[1]), reverse=True)
    cur_sum = 0
    important_weights = []
    for w in non_zero_weights:
        cur_sum += abs(w[1])
        important_weights.append(w)
        if cur_sum >= percentage * total_weight:
            break
    return np.array(important_weights)

In [10]:
from sklearn.linear_model import LogisticRegression

liblinear_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.linspace(0.1, 10, 100)
 }
param_grid = [liblinear_param_grid]
logistic_model = LogisticRegression(solver='liblinear')
logistic_grid_search = my_grid_search(logistic_model, param_grid, X_train_scaled, Y_train, X_test_scaled, Y_test)
logistic_important_weights = important_features(logistic_grid_search.best_estimator_.coef_[0], X.columns, percentage=0.8)
logistic_important_weights

{'C': 1.0, 'penalty': 'l1'}
Train best accuracy: 0.9045977011494253
Test accuracy: 0.8412698412698413


array([['VSA_EState4', '2.1158226510661007'],
       ['VSA_EState9', '-1.353953731979301'],
       ['FpDensityMorgan3', '1.1604002451718354'],
       ['fr_ArN', '-0.9829364762758019'],
       ['fr_priamide', '0.8655383056587539'],
       ['fr_Ar_COO', '0.6890276476863252'],
       ['VSA_EState7', '-0.6775738393578046'],
       ['fr_quatN', '0.6725893441788177'],
       ['SMR_VSA4', '0.635220218504794'],
       ['fr_ketone_Topliss', '0.6011280502344001'],
       ['fr_NH2', '-0.5046400019629055'],
       ['fr_azide', '-0.43252087919574184'],
       ['fr_halogen', '0.42878237058033664'],
       ['MaxEStateIndex', '-0.4035656982260478'],
       ['SMR_VSA3', '-0.36644249328355377'],
       ['fr_NH0', '-0.35800879604530433']], dtype='<U32')

In [11]:
from sklearn.svm import SVC

param_grid = { 
    'C': np.linspace(0.01, 10, 100)
 }
svm_model = SVC(kernel = 'linear')
svm_grid_search = my_grid_search(svm_model, param_grid, X_train_scaled, Y_train, X_test_scaled, Y_test)
svm_important_weights = important_features(svm_grid_search.best_estimator_.coef_[0], X.columns, percentage=0.8)
svm_important_weights

{'C': 0.21181818181818182}
Train best accuracy: 0.8705747126436781
Test accuracy: 0.8253968253968254


array([['VSA_EState4', '0.7281063564082668'],
       ['fr_priamide', '0.44753092311013587'],
       ['FpDensityMorgan3', '0.41402227424708127'],
       ['fr_NH2', '-0.3965792602335351'],
       ['NHOHCount', '0.38177261186826705'],
       ['fr_methoxy', '-0.33736697532042303'],
       ['fr_Ar_COO', '0.33088386883461485'],
       ['fr_quatN', '0.32346232698192007'],
       ['fr_ArN', '-0.3215890915166288'],
       ['FpDensityMorgan2', '0.3059444107535855'],
       ['MaxAbsEStateIndex', '-0.29410921141170276'],
       ['MaxEStateIndex', '-0.29410921141170276'],
       ['VSA_EState7', '-0.26288676433146974'],
       ['PEOE_VSA8', '-0.2610344158832696'],
       ['FractionCSP3', '-0.2561401751584095'],
       ['SMR_VSA4', '0.23882699071440755'],
       ['SlogP_VSA8', '0.23486510391883475'],
       ['fr_piperzine', '0.2347812185265488'],
       ['NumSaturatedHeterocycles', '-0.22568544818583688'],
       ['fr_allylic_oxid', '-0.2216692701290928'],
       ['VSA_EState9', '-0.22032209461772007

In [12]:
from sklearn.ensemble import RandomForestClassifier

param_grid = { 
    'n_estimators': [10, 50, 100, 300],
    'max_samples': np.linspace(0.1, 1, 10),
 }
forest_model = RandomForestClassifier(random_state=38, bootstrap=True, oob_score=True, max_features='sqrt')
forest_grid_search = my_grid_search(forest_model, param_grid, X_train_scaled, Y_train, X_test_scaled, Y_test)
forest_important_weights = important_features(forest_grid_search.best_estimator_.feature_importances_, X.columns, percentage=0.8)
forest_important_weights

/home/coder/.conda/envs/molecules/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/home/coder/.conda/envs/molecules/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/home/coder/.conda/envs/molecules/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/home/coder/.conda/envs/molecules/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/home/coder/.conda/envs/molecules/lib/python3.8/site-package

{'max_samples': 0.5, 'n_estimators': 50}
Train best accuracy: 0.8836781609195402
Test accuracy: 0.873015873015873


array([['Kappa1', '0.03857126549801988'],
       ['Chi1', '0.03549553472261597'],
       ['EState_VSA8', '0.027460582604645106'],
       ['LabuteASA', '0.02692281104635348'],
       ['PEOE_VSA9', '0.024418734409056838'],
       ['Chi1v', '0.024215585602233258'],
       ['EState_VSA3', '0.022278979306482788'],
       ['Chi2v', '0.0220006117075989'],
       ['Chi0n', '0.021782457582906048'],
       ['Kappa2', '0.02111930987602531'],
       ['Chi2n', '0.019877151249761447'],
       ['NHOHCount', '0.019814518792669614'],
       ['VSA_EState6', '0.019312830893997893'],
       ['MinPartialCharge', '0.01853729919552752'],
       ['EState_VSA10', '0.017494754099823758'],
       ['HeavyAtomMolWt', '0.01683308872500049'],
       ['EState_VSA5', '0.015639318327342975'],
       ['NumHeteroatoms', '0.01462907369597661'],
       ['NumHAcceptors', '0.014257618803758351'],
       ['BertzCT', '0.014077661081180137'],
       ['fr_quatN', '0.013120469805671523'],
       ['BalabanJ', '0.01302442745984602'

In [13]:
from xgboost import XGBClassifier

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'learning_rate': np.linspace(0.01, 2, 10),
    'reg_alpha': np.linspace(0, 1, 5),
    'reg_lambda': np.linspace(0, 1, 5),
}

xgb_model = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss', max_depth=3, max_leaves=8, subsample = 0.7, colsample_bynode = 0.7)
gb_grid_search = my_grid_search(xgb_model, param_grid, X_train_scaled, Y_train, X_test_scaled, Y_test)
gb_important_weights = important_features(gb_grid_search.best_estimator_.feature_importances_, X.columns, percentage=0.8)
gb_important_weights

{'learning_rate': 0.23111111111111113, 'n_estimators': 10, 'reg_alpha': 0.75, 'reg_lambda': 0.0}
Train best accuracy: 0.8839080459770117
Test accuracy: 0.9365079365079365


array([['MinEStateIndex', '0.114264525'],
       ['Chi1', '0.10593105'],
       ['Kappa2', '0.08011054'],
       ['EState_VSA3', '0.053387117'],
       ['SMR_VSA7', '0.052988943'],
       ['NHOHCount', '0.052649867'],
       ['Chi0v', '0.043448832'],
       ['PEOE_VSA8', '0.04148734'],
       ['SlogP_VSA1', '0.0363599'],
       ['VSA_EState9', '0.032875553'],
       ['VSA_EState4', '0.03234256'],
       ['Chi4v', '0.0317564'],
       ['qed', '0.029430768'],
       ['fr_quatN', '0.02651104'],
       ['MolLogP', '0.022995561'],
       ['SlogP_VSA4', '0.02187782'],
       ['MaxAbsEStateIndex', '0.021279952'],
       ['PEOE_VSA10', '0.020244898']], dtype='<U32')

In [14]:
logistic_important_weights_set = set(logistic_important_weights[:, 0])
svm_important_weights_set = set(svm_important_weights[:, 0])
forest_important_weights_set = set(forest_important_weights[:, 0])
gb_important_weights_set = set(gb_important_weights[:, 0])
intersection_logistic_svm = logistic_important_weights_set.intersection(svm_important_weights_set)
display(intersection_logistic_svm)

intersection_forest_gb = forest_important_weights_set.intersection(gb_important_weights_set)
display(intersection_forest_gb)

intersection_all = intersection_logistic_svm.intersection(intersection_forest_gb)
display(intersection_all)

{'FpDensityMorgan3',
 'MaxEStateIndex',
 'SMR_VSA3',
 'SMR_VSA4',
 'VSA_EState4',
 'VSA_EState7',
 'VSA_EState9',
 'fr_ArN',
 'fr_Ar_COO',
 'fr_NH0',
 'fr_NH2',
 'fr_azide',
 'fr_halogen',
 'fr_ketone_Topliss',
 'fr_priamide',
 'fr_quatN'}

{'Chi0v',
 'Chi1',
 'EState_VSA3',
 'Kappa2',
 'MinEStateIndex',
 'MolLogP',
 'NHOHCount',
 'SMR_VSA7',
 'SlogP_VSA1',
 'VSA_EState4',
 'fr_quatN',
 'qed'}

{'VSA_EState4', 'fr_quatN'}